In [0]:
%python
# 05_feature_building_candidates_v2.py
import json
from pyspark.sql import functions as F
from trimble_geospatial_demo_utils.site_lock import acquire_site_lock, release_site_lock
from trimble_geospatial_demo_utils import send_notification

# ==================================================
# Unity Catalog context
# ==================================================
spark.sql("USE CATALOG main")
spark.sql("USE SCHEMA demo")

# ==================================================
# Read Job Parameters
# ==================================================
dbutils.widgets.text("siteId", "", "Site ID")
dbutils.widgets.text("uploadJobId", "", "Upload Job ID")
dbutils.widgets.text("notificationUrl", "", "Notification URL")
dbutils.widgets.text("dbxWebhookSecret", "", "DBX Webhook Secret")

SITE_ID = dbutils.widgets.get("siteId")
UPLOAD_JOB_ID = dbutils.widgets.get("uploadJobId")
NOTIFICATION_URL = dbutils.widgets.get("notificationUrl")
DBX_WEBHOOK_SECRET = dbutils.widgets.get("dbxWebhookSecret")

# Validate required parameters
if not SITE_ID:
    raise ValueError("Missing required job parameter: siteId")

print(f"🏗️  Site ID: {SITE_ID}")

# ==================================================
# CONFIG
# ==================================================
PATCHES_TABLE = "surface_patches_v2"
OUTPUT_TABLE  = "features_building_candidates_v2"
OUTPUT_PATH   = "abfss://processed@trimblegeospatialdemo.dfs.core.windows.net/features_building_candidates_v2"

# Thresholds (tunable)
MIN_PATCH_AREA_M2       = 20.0
MIN_HEIGHT_ABOVE_GROUND = 2.5
MAX_HEIGHT_RANGE        = 3.0
MIN_POINTS_USED         = 200

JOB_RUN_ID = spark.conf.get("spark.databricks.job.runId", "manual")

# ==================================================
# Acquire site lock
# ==================================================
acquire_site_lock(
    spark=spark,
    site_id=SITE_ID,
    locked_by=JOB_RUN_ID,
    ttl_minutes=90
)

try:
    # ==================================================
    # 1) Load surface patches (ground only)
    # ==================================================
    df_patches = (
        spark.table(PATCHES_TABLE)
             .filter(F.col("siteId") == SITE_ID)
             .filter(F.col("surfaceType") == F.lit("ground"))
    )

    if df_patches.rdd.isEmpty():
        raise RuntimeError("No surface patches found")

    # ==================================================
    # 2) Estimate local ground reference per tile
    #    (low percentile ground height)
    # ==================================================
    df_ground_ref = (
        df_patches
        .groupBy("siteId", "tileId")
        .agg(
            F.expr("percentile_approx(minZ, 0.1)").alias("groundZ_ref")
        )
    )

    df = (
        df_patches
        .join(df_ground_ref, ["siteId", "tileId"], "left")
        .withColumn("heightAboveGround", F.col("meanZ") - F.col("groundZ_ref"))
        .withColumn("heightRange", F.col("maxZ") - F.col("minZ"))
    )

    # ==================================================
    # 3) Candidate filtering (rule-based)
    # ==================================================
    df_candidates = (
        df
        .filter(F.col("pointsUsed") >= F.lit(MIN_POINTS_USED))
        .filter(F.col("heightAboveGround") >= F.lit(MIN_HEIGHT_ABOVE_GROUND))
        .filter(F.col("heightRange") <= F.lit(MAX_HEIGHT_RANGE))
    )

    # ==================================================
    # 4) Compute final features
    # ==================================================
    df_features = (
        df_candidates
        .select(
            "siteId",
            "tileId",
            F.concat_ws("|", F.col("tileId"), F.col("patchId")).alias("buildingCandidateId"),
            "pointsUsed",
            "cellCount",
            "minZ",
            "meanZ",
            "maxZ",
            "heightAboveGround",
            "heightRange"
        )
        .withColumn("computedAt", F.current_timestamp())
    )

    # ==================================================
    # 5) Write latest snapshot by site
    # ==================================================
    (
        df_features.write
            .format("delta")
            .mode("overwrite")
            .option("replaceWhere", f"siteId = '{SITE_ID}'")
            .option("path", OUTPUT_PATH)
            .partitionBy("siteId", "tileId")
            .saveAsTable(OUTPUT_TABLE)
    )

    print("✅ features_building_candidates_v2 written successfully")
except Exception as e:
    if NOTIFICATION_URL and DBX_WEBHOOK_SECRET:
        payload = {
            "runId": spark.conf.get("spark.databricks.job.runId", "manual-notebook"),
            "jobId": UPLOAD_JOB_ID,
            "status": "FAILED",
            "error": str(e),
            "siteId": SITE_ID,
        }
        try:
            send_notification(json.dumps(payload), NOTIFICATION_URL, webhook_secret=DBX_WEBHOOK_SECRET)
        except Exception as notify_ex:
            print("⚠️ Notification failed:", str(notify_ex)[:200])
    raise
finally:
    release_site_lock(
        spark=spark,
        site_id=SITE_ID,
        locked_by=JOB_RUN_ID,
    )